### 사진 화이트 밸런스 변환기
    
    최초 작성일 : 2/9
    작성자 : 양희승
    
    내용 : 사진의 빛이 다르기 때문에 하나로 정리할 화이트 밸런스 변환하는 코드
    
    현재 테스트 단계

In [2]:
# 설치
#  sudo apt-get install python-opencv  
# !pip install opencv-python  #메인모듈만
# !pip install opencv-contrib-python  #기본 및 contrib 모듈이 모두 필요한 경우

# 임포트가 안된다면
# sudo -H python -m pip install opencv-python 
import cv2 as cv
import numpy as np

In [3]:
def show(final):
    print('display')
    cv.imshow("Temple", final)
    cv.waitKey(0)
    cv.destroyAllWindows()

def saveimg(final) :
    print('saving')
    cv.imwrite("result.jpg", final)

In [1]:
# #Insert anyfilenmae with path
# img = cv.imread("img/1.jpg")
# res = img
# final = cv.cvtColor(res, cv.COLOR_BGR2LAB)


# avg_a = -np.average(final[:,:,1])
# avg_b = -np.average(final[:,:,2])

# for x in range(final.shape[0]) :
#     for y in range(final.shape[1]) :
#         l, a, b = final[x][y]
#         shift_a =avg_a * (1/100.0) * 1.1
#         shift_b =avg_b * (1/100.0) * 1.1
#         final[x][y][1]= a + shift_a
#         final[x][y][2]= b + shift_b
        
# final = cv.cvtColor(final, cv.COLOR_LAB2BGR)
# final = np.hstack((res, final))
# show(final)
# saveimg(final)


In [4]:
def wb_conversion(img) :
    result = cv.cvtColor(img, cv.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv.cvtColor(result, cv.COLOR_LAB2BGR)
    return result

In [14]:
#Insert anyfilenmae with path
img = cv.imread("img/5.jpg")
wb_img = wb_conversion(img)
h, w, channel = wb_img.shape
print("높이 : ",h)
print("가로 : ",w)
print("채널 : ",channel)
if h > w :
    final = cv.resize(wb_img, dsize=(540, 720), interpolation=cv.INTER_AREA)
elif h < w :
    final = cv.resize(wb_img, dsize=(720, 540), interpolation=cv.INTER_AREA)
else :
    final = cv.resize(wb_img, dsize=(540, 540), interpolation=cv.INTER_AREA)

cv.imwrite('img/5_convert.jpg', final)

높이 :  1080
가로 :  1440
채널 :  3


True